# hands-on implementation of cnn with numpy--------pooling layer

## pooling layer meaning
- 平移不变形 more robust
- dimensionality reduction (more efficient)
- 只关注相对位置的时候 可以采用 实现尺度变化(subsampling)

> NOTE:pooling layer已经可以通过**stride!=1 conv2d**实现subsampling代替

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
print(np.__version__)

1.16.4


实际上pooling layer没有参与前向计算的参数的，
所以我们只需要实现 $$\frac{\partial { loss}}{\partial Pooling\_in} = Pooling.backward(\frac{\partial { loss}}{\partial Pooling\_out} )$$

## MaxPooling layer

In [22]:
class MaxPooling(object):
    def __init__(self,shape,ksize,stride):
        self.input_shape = shape
        self.ksize = ksize
        self.stride = stride
        self.input_channels = shape[0]
        self.output_channels = shape[-1]
        # 生效位置索引
        self.index = np.zeros(shape)
        self.output_shape = [shape[0],shape[1]//ksize,shape[2]//ksize,shape[-1]]
    
    def forward_propagate(self,x):
        # calculate and record index
        shape = x.shape
        out = np.zeros([shape[0],shape[1]//self.stride,
                       shape[2]//self.stride,shape[-1]])
        self.index = np.zeros(self.input_shape)
        for b in range(shape[0]):
            for c in range(shape[-1]):
                for i in range(0,shape[1],self.stride):
                    for j in range(0,shape[2],self.stride):
                        if i+self.ksize > self.input_shape[1] or j+self.ksize > self.input_shape[1]:

                            continue
                        else:
                            out[b,i//self.stride,j//self.stride,c] = np.max(
                                    x[b,i:i+self.stride,j:j+self.stride,c])
                            idx = np.argmax(x[b,i:i+self.stride,j:j+self.stride,c])
                            self.index[b,i + idx//self.ksize,j + idx%self.ksize,c] += 1
        return out
    
    def backward_propagate(self,delta):
        # 进行扩充 之后与idx进行点乘
        repeated_delta = np.repeat(np.repeat(delta,self.stride,axis=1),self.stride,axis=2)
        return repeated_delta * self.index


In [ ]:
# demo
a = [[[1,2],[3,4]],[[10,2],[31,4]]]
print(a)
print(np.argmax(a))
np.repeat([[1,2],[3,4]],repeats=2,axis=1)

## AveragePooling layer

In [23]:
class AveragePooling(object):
    def __init__(self,shape,ksize,stride):
        self.input_shape = shape
        self.ksize = ksize
        self.stride = stride
        self.input_channels = shape[0]
        self.output_channels = shape[-1]
        self.output_shape = [shape[0],shape[1]//ksize,shape[2]//ksize,shape[-1]]
    
    def forward_propagate(self,x):
        # calculate and record index
        shape = x.shape
        out = np.zeros([shape[0],shape[1]//self.stride,
                       shape[2]//self.stride,shape[-1]])
        
        for b in range(shape[0]):
            for c in range(shape[-1]):
                for i in range(0,shape[1],self.stride):
                    for j in range(0,shape[2],self.stride):
                        if i+self.ksize > self.input_shape[1] or j+self.ksize > self.input_shape[1]:
                            continue
                        else:
                            out[b,i//self.stride,j//self.stride,c] = np.mean(
                                    x[b,i:i+self.stride,j:j+self.stride,c])

        return out
    
    def backward_propagate(self,delta):
        # 进行扩充 之后与idx进行点乘
        repeated_delta = np.repeat(np.repeat(delta,self.stride,axis=1),self.stride,axis=2)
        return repeated_delta / (self.ksize*self.ksize)
    

In [30]:
# test code
if __name__ == '__main__':
    img = np.random.randn(5,32,32,3)
    pool1 = MaxPooling(img.shape,2,2)
    pool2 = AveragePooling(img.shape,2,2)
    out1 = pool1.forward_propagate(img)
    out2 = pool2.forward_propagate(img)
    delta1 = pool1.backward_propagate(out1*2)
    delta2 = pool2.backward_propagate(out2*2)
    print(out1.shape,out2.shape,delta1.shape,delta2.shape)
    print(pool1.index.shape,pool2.index.shape)
    print(np.max(pool1.index),np.max(delta2),np.max(out2))

(5, 16, 16, 3) (5, 16, 16, 3) (5, 32, 32, 3) (5, 32, 32, 3)
(5, 32, 32, 3) (5, 32, 32, 3)
1.0 0.9004585265128483 1.8009170530256966
